# Introduction

## Overview of Problem

We have received Vanderbilt hospital data extracted from the Synthetic Derivative. At Vanderbilt, bioinformaticians helped to create a "mirror image" of electronic medical records such as those in BioVU, Vanderbilt's biorepository of DNA extracted from discarded blood collected during routine clinical testing. This mirror of the EMR is called the Synthetic Derivative, and it contains over 2 million individual patients with all clinical information available for the past 10 years. It has been scrubbed of HIPAA identifiers with an eror rate of ~0.01%, meaning that the data has been deidentified with a subject ID. 

The objective of this project is to employ modeling tools introduced in class to fit prediction models for patient readmission within 30 days of discharge using given data. The data includes multiple variables, detailed below in "Data." Ideally, our model will be able to well predict readmission within 30 days of discharge for a patient using the admit/discharge/transfer events data. The rest of the data detials information about the patients themselves, tests and treatments they underwent at Vanderbilt, and lab results and medication. Using these variables, we hope to accurately predict readmission. This information could be very useful for actually clinicians hoping to predict which patients may need to be readmitted, and which characteristics/tests cause them to be readmitted. Once this is determined, that subset of patients could be given more attention and/or tests to prevent readmission.

## Goal and Structure of Project

This project will introduce several approaches to predictive modeling of patient readmission within 30 days of discharge. Three approaches are detailed in the following jupyter notebooks, each of which may include more than one modeling type, attempts to improve each model and test performance, and tuning of the models to increase the goodness of fit. Cross-validation will be used when appropriate, and model selection methods and/or explanations of the models chosen will be provided for each notebook. We will then justify and describe each model selection, and provide visualizations and discussions of the results. The models will be compared using goodness of fit tests and other performance characteristics. For clarification, the steps for each model notebook are listed below, and enumerated in the following 3 modeling notebooks. 

1. Identify the model approach(es), describe, and justify the selection
2. Code, parameterize, and run model (including visualization)
3. Cross-validation
4. Goodness of fit assessments, performance characteristics (including visualization)
5. Improvements to model/tuning of parameters; model selection methods, justification of improvements/tests
6. Comparison of models; identification of best model
7. Results 
8. Implications of model and conclusions

We have also included a conclusions notebook that details the comparisons of the 3 model types, which ones worked and didn't work, our best model, and future directions.

## Data and cleaning

In [2]:
import statsmodels.formula.api as smf
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy  as np
import csv

### ADT: 
Admit/Discharge/Transfer Events. Includes the variables:
1. "Event" (Admit, Transfer, or Discharge)
2. "Admission_date" (date format, M/DD/YY)
3. "Event_Date" (date)
4. "SRV_CODE" (e.g. ORT, NEU, GMB)
5. "CHIEF_COMPLAINT" (e.g. CP, CHEST PAIN, SEIZURES)
6. "DISCHARGE_DATE" (date)

This dataframe was cleaned by parsing the dates from strings to pandas datetime format, replacing column names and making everything lowercase for easier data parsing. The data is a little redundant; each row has an "event," either an admit, transfer, or discharge, but also has the admission and discharge dates for that entire stay for that patient. In other words, the admission and discharge dates are repeated on multiple rows for the same stay: at least two rows (one for admit and one for discharge) but possibly more, depending on the number of transfers. We collapsed these rows so that each hospital stay is represented by only one row. Patient IDs may be repeated, depending on readmission rates.

This data frame was used to organize the variables for each patient for each hospital stay, as well as to generate a variable for prediction. This "y" variable dataframe was generated by looking at one patient and their admission and discharge dates. If the patient had any admission dates within 30 days of a discharge date, this variable is given the value "1"; if there were none, the patient got a value of "0."

In [24]:
data_path = "/Users/geenaildefonso/Downloads/DataP/"

In [3]:
#   imports & variables
###
data_path = "/Users/geenaildefonso/Downloads/DataP/"


###
#   ADT
### 
adt = pd.read_csv(data_path+"FONNESBECK_ADT.csv", na_values=[''], 
                  parse_dates=['Admission_date', 'Event_Date', 'DISCHARGE_DATE'],
                  encoding = "ISO-8859-1")
adt.head()

# rename the columns and replace event strings with simpler versions
# #todo -- expand categorical variables using get_dummies
adt_clean = (adt
             .rename(columns={"RUID": "patient_id", 
                              "Event":"adt_event", 
                              "Admission_date": "admission_date",
                              "Event_Date": "adt_event_date", 
                              "SRV_CODE": "srv_code",
                              "CHIEF_COMPLAINT": "chief_complaint", 
                              "DISCHARGE_DATE": "discharge_date"})
             .replace({'adt_event': {'.*Admit': 'admit',
                                     '.*Discharge': 'discharge', 
                                     '.*Transfer': 'transfer'}}, regex=True))

adt_clean.head()

# calculate the amount of missing data in the ADT table
adt_clean.isnull().sum()

# presumably only discharges will have discharge dates; these actual missing data
(adt_clean[adt_clean.adt_event == 'discharge']).isnull().sum()

patient_id           0
adt_event            0
admission_date       2
adt_event_date       0
srv_code             0
chief_complaint    265
discharge_date       0
dtype: int64

Since there are only two missing admission dates, we think we can safely exclude this data. We are not using the chief_complaint because there is a wide range of complaints that are not necessarily related to the hospital stay, and can change for each hospital stay.

In [4]:
df = pd.DataFrame(adt_clean, columns = ['patient_id','adt_event','admission_date', 'discharge_date'])
df = df[df.adt_event != 'transfer']
df = df[df.adt_event != 'discharge']
# adt_final
df_adt = df[['patient_id','admission_date', 'discharge_date']]
df_adt = df_adt.sort_values('admission_date')
df_adt = df_adt.sort_values('patient_id')
df_adt=df_adt.reset_index(drop=True)

df_adt.to_csv("df_adt_clean.csv")
df_adt.head()

,patient_id,admission_date,discharge_date
0,50135262,2007-02-08,2007-02-12
1,50135262,2007-08-03,2007-08-06
2,50135262,2014-11-15,2014-11-18
3,50135262,2007-08-28,2007-08-29
4,50135262,2012-09-15,2012-09-22


### BMI: 
Body mass index measurement information.
Includes the variables:
1. "BMI" (numeric)
2. Date_BMI (date M/DD/YY)
3. BMI_Weight (numeric, in kg)
4. BMI_Height (numeric, in cm)
5. Pregnancy_Indicator (0, 1).

Cleaning of the BMI dataset: We decided that BMI gave us enough information, and height and weight were a little superfluous. In order to reduce the number of possible variables in the model, we only included BMI, the date the BMI measurement was taken, and a pregnancy indicator in the final dataset for modeling. We used "Date_BMI" to include this data into our larger "X" dataset by checking where this date fell between the patient's admission and discharge dates and adding the data to that row.

In [13]:
###
#   BMI
###
bmi = pd.read_csv(data_path+"FONNESBECK_BMI.csv", parse_dates=['Date_BMI'], infer_datetime_format=True)
bmi.head()

bmi_clean = (bmi
             .rename(columns={"RUID": "patient_id", 
                              "BMI": "bmi",
                              "Date_BMI": "bmi_date", 
                              "BMI_Weight": "weight",
                              "BMI_Height": "height", 
                              "Pregnancy_Indicator": "pregnant"}))
bmi_clean['bmi_date'] = pd.to_datetime(bmi_clean.bmi_date, errors='coerce')
bmi_clean.head()

# small amount of missingness
# #todo -- possible to fill in missing if same patient
# bmi_clean.isnull().sum()
df_bmi = bmi_clean.drop_duplicates(subset='bmi_date')
df_bmi = df_bmi[['patient_id', 'bmi', 'bmi_date','pregnant']].dropna()
df_bmi.head()
# bmi_clean.groupby('bmi_date',sort=True).sum()

,patient_id,bmi,bmi_date
0,50135262,41.43,2005-01-09
1,50135262,22.86,2011-02-11
2,50135262,43.07,2011-02-12
5,50135262,41.13,2011-02-13
7,50135262,40.29,2011-02-14


In [53]:
unique_ids = df_bmi.patient_id.unique()
ps_means = list(range(len(unique_ids)))
ps_std = list(range(len(unique_ids)))
ps_trends = list(range(len(unique_ids)))
# pd_trends = list(range(len(unique_ids)))
for p, count in zip(unique_ids, range(len(unique_ids))):
#make a subsetted dataframe of just that ID
    p_df = df_bmi[df_bmi.patient_id == p]
    p_df=p_df.reset_index(drop=True)
    #for each measurement in that list:
    s_mean=p_df.bmi.mean()
    s_std=p_df.bmi.std()
#     d_mean=p_df.diastolic.mean()
    ps_means[count]=s_mean
    ps_std[count] = s_std
#     pd_means[count]=d_mean
    #Trends in dp and sp: fit each patient's data with a linear regression and record the slope.
    d_regr = linear_model.LinearRegression()
    d_regr.fit(p_df.bmi_date.reshape(-1, 1), p_df.bmi.reshape(-1, 1))
    b=d_regr.coef_.item(0)
    ps_trends[count]=b

x_bmi = pd.DataFrame({'patient_id':unique_ids, 
                     'bmi_mean': ps_means,'bmi_std': ps_std})

/Users/geenaildefonso/anaconda/envs/bios8366/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [54]:
x_bmi = x_bmi.fillna(value=0)
x_bmi

,bmi_mean,bmi_std,patient_id
0,43.443846,4.362163,50135262
1,40.104062,56.286141,50135361
2,29.091786,26.354793,50135369
3,26.887687,2.139820,50135375
4,33.648269,5.109400,50135425
5,26.157614,12.252343,50135437
6,23.426466,4.507067,50135624
7,23.594615,1.965764,50135735
8,30.613953,2.414127,50135759
9,29.951538,1.140474,50135776


### BP: 
Blood pressure measurements. Includes the variables:
1. "SYSTOLIC" (integer)
2. "DIASTOLIC" (integer)
3. "Measure_date" (M/DD/YY)

The Systolic and Diastolic variables are used in the "X" dataset by taking a mean over all of the patients' values and recording one systolic pressure measurement and one diastolic pressure measurement per patient. We also included BPS_trend and BPD_trend variables, which is the slope of a linear regression model fit to the BP pressures over time. However, the maximum value for the trends was still very close to 0-- about 10^-13. Thus, we will have a general, mean measurement describing each patient, and will remove the trend values, since each patient's either stays constant or does not show a signficant increasing or decreasing trend.


In [ ]:
###
#   BP
###
bp = pd.read_csv(data_path+"FONNESBECK_BP.csv", parse_dates=['Measure_date'], infer_datetime_format=True)
bp.head()

bp_clean = (bp
            .rename(columns={"RUID": "patient_id", 
                             "SYSTOLIC": "systolic",
                             "DIASTOLIC": "diastolic", 
                             "Measure_date": "bp_date"}))
bp_clean['bp_date'] = pd.to_datetime(bp_clean.bp_date, errors='coerce')

# only missing dates, may not need to address
bp_clean.isnull().sum()

In [ ]:
bp_clean = bp_clean.sort_values('patient_id')
bp_clean=bp_clean.reset_index(drop=True)

bp_clean.to_csv("bp_clean.csv")
bp_clean.head()

In [27]:
bp_clean = pd.read_csv(data_path+"df_bp_clean.csv", parse_dates=['bp_date'], infer_datetime_format=True, index_col =0)
bp_clean.head()

/Users/geenaildefonso/anaconda/envs/bios8366/lib/python3.6/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,patient_id,systolic,diastolic,bp_date
0,50135262,150,80,2005-01-09
1,50135262,137,77,2007-02-08
2,50135262,137,78,2007-02-08
3,50135262,190,78,2007-02-08
4,50135262,190,77,2007-02-08


In [28]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

unique_ids = bp_clean.patient_id.unique()
ps_means = list(range(len(unique_ids)))
pd_means = list(range(len(unique_ids)))
ps_trends = list(range(len(unique_ids)))
pd_trends = list(range(len(unique_ids)))
for p, count in zip(unique_ids, range(len(unique_ids))):
#make a subsetted dataframe of just that ID
    p_df = bp_clean[bp_clean.patient_id == p]
    p_df=p_df.reset_index(drop=True)
    #for each measurement in that list:
    s_mean=p_df.systolic.mean()
    d_mean=p_df.diastolic.mean()
    ps_means[count]=s_mean
    pd_means[count]=d_mean
    #Trends in dp and sp: fit each patient's data with a linear regression and record the slope.
    d_regr = linear_model.LinearRegression()
    d_regr.fit(p_df.bp_date.reshape(-1, 1), p_df.diastolic.reshape(-1, 1))
    b=d_regr.coef_.item(0)
    pd_trends[count]=b

x_bp = pd.DataFrame({'patient_id':unique_ids, 
                     'ps_mean': ps_means, 'pd_mean': pd_means,
                     'ps_trend':ps_trends, 'pd_trend':pd_trends})


/Users/geenaildefonso/anaconda/envs/bios8366/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [30]:
x_bp.head()

,patient_id,pd_mean,pd_trend,ps_mean,ps_trend
0,50135262,64.086464,-6.671398e-18,141.160834,0
1,50135361,59.877885,1.522614e-17,112.785374,1
2,50135369,70.011200,-1.273555e-17,115.893943,2
3,50135375,67.844574,-6.963159e-17,118.336822,3
4,50135425,56.765079,-8.611786e-17,128.038095,4


As mentioned above, the largest pd_trend value is about 10^-13. This was investigated for one patient below, showing that even though each blood pressure measurement varied, it was not in a significant direction. Thus, we decided to include the variance instead of the trend (slope of the linear regression).

In [ ]:
bp_clean.loc[bp_clean.patient_id == 50135262]

In [31]:
import statsmodels.formula.api as smf
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

unique_ids = bp_clean.patient_id.unique()
ps_means = list(range(len(unique_ids)))
pd_means = list(range(len(unique_ids)))
ps_variance = list(range(len(unique_ids)))
pd_variance = list(range(len(unique_ids)))
for p, count in zip(unique_ids, range(len(unique_ids))):
#make a subsetted dataframe of just that ID
    p_df = bp_clean[bp_clean.patient_id == p]
    p_df=p_df.reset_index(drop=True)
    #for each measurement in that list:
    s_mean=p_df.systolic.mean()
    d_mean=p_df.diastolic.mean()
    ps_means[count]=s_mean
    pd_means[count]=d_mean
    pd_variance[count]=p_df.diastolic.std()
    ps_variance[count]=p_df.systolic.std()

In [32]:
bp_full_clean = pd.DataFrame({'patient_id':unique_ids, 
                     'ps_mean': ps_means, 'pd_mean': pd_means,
                     'ps_std':ps_variance, 'pd_std':pd_variance})

In [34]:
x_bp = bp_full_clean.fillna(value=0)
x_bp.head()

,patient_id,pd_mean,pd_std,ps_mean,ps_std
0,50135262,64.086464,13.579633,141.160834,20.377205
1,50135361,59.877885,12.678492,112.785374,18.694088
2,50135369,70.011200,11.986641,115.893943,17.587347
3,50135375,67.844574,12.547771,118.336822,21.254275
4,50135425,56.765079,9.300600,128.038095,16.444496


### MED: 
Medications information, including dose and duration.
Includes the variables:
1. "Entry_Date" (date M/DD/YY)	
2. "Drug_Name" (common drug name, string)	
3. "DRUG_FORM" (if drug comes in multiple forms, this describes which form is given. E.g. nebulizer versus inhaler for albuterol)	
4. "DRUG_STRENGTH" (mL, or NA)
5. "Route" (Route of drug administration; e.g. IV, FLUSH, PO)
6. "Dose_Amt" (Amount of drug, variable units; g, ML/HR, units)
7. "Drug_Freq" (number of times given/how the drug is given; e.g. twice daily, once, Q1H PRN)
8. "Duration" (length of time drug is given; e.g. months, days, etc)

Cleaning of the MED dataset: From this dataset, we used Entry_date to include the data in our larger "X" data set. We included Drug_Name and dose amount, combined to be one variable. No other information from this dataset was used.

In [ ]:
###
#   MED
###
med = pd.read_csv(data_path+"FONNESBECK_MED.csv", parse_dates=['Entry_Date'], infer_datetime_format=True)
med.head()

med_clean = (med
             .rename(columns={"RUID": "patient_id", 
                              "Entry_Date": "drug_entry_date",
                              "Drug_Name": "drug_name", 
                              "DRUG_FORM": "drug_form",
                              "DRUG_STRENGTH": "drug_strength",
                              "Route": "drug_route",
                              "Dose_Amt": "drug_dose",
                              "Drug_Freq": "drug_freq",
                              "Duration": "drug_duration"}))
med_clean.head()

# lots of missing data in this table
# many cols likely uninformative; drug_name might be most useful
med_clean.isnull().sum()

### CPT: 
Procedure codes. Variables include:
1. "CPT_code" (integer in most cases to describe categorical data)
2. "Event_date" (M/DD/YY)

This data was included in models that could predict using categorical data.

In [ ]:
###
#   CPT
###
cpt = pd.read_csv(data_path+"FONNESBECK_CPT.csv", parse_dates=['Event_date'], infer_datetime_format=True)
cpt.head()

cpt_clean = (cpt
             .rename(columns={"RUID": "patient_id", 
                              "CPT_Code": "cpt_code",
                              "Event_date": "cpt_event_date"}))
cpt_clean.head()

# no missing data
cpt_clean.isnull().sum()

### EGFR: 
Estimated Glomerular Filtration Rate measurements, used to screen for kidney damage. Obtained from a creatine lab. Includes the variables:
1. "EGFR" (continuous data)
2. "egfr_date" (date M/DD/YY)

Again, we included this in the X dataset.

In [37]:
###
#   EGFR
###
egfr = pd.read_csv(data_path+"df_egfr_clean.csv", parse_dates=['egfr_date'], infer_datetime_format=True)
egfr.head()

# egfr_clean = (egfr
#               .rename(columns={"RUID": "patient_id", 
#                                "EGFR": "egfr"}))
# egfr_clean.head()

# no missing data
egfr_clean.isnull().sum()
df_egfr = egfr_clean[['patient_id', 'egfr', 'egfr_date']]
df_egfr.head()

,patient_id,egfr,egfr_date
0,50135262,123.68000,2007-02-08
1,50135262,123.67783,2007-02-08
2,50135262,76.40173,2011-02-11
3,50135262,76.40000,2011-02-11
4,50135262,78.64000,2011-02-12


In [84]:
unique_ids = df_egfr.patient_id.unique()
ps_means = list(range(len(unique_ids)))
ps_std = list(range(len(unique_ids)))
ps_trends = list(range(len(unique_ids)))
# pd_trends = list(range(len(unique_ids)))
for p, count in zip(unique_ids, range(len(unique_ids))):
#make a subsetted dataframe of just that ID
    p_df = df_egfr[df_egfr.patient_id == p]
    p_df=p_df.reset_index(drop=True)
    #for each measurement in that list:
    s_mean=p_df.egfr.mean()
    s_std=p_df.egfr.std()
#     d_mean=p_df.diastolic.mean()
    ps_means[count]=s_mean
    ps_std[count] = s_std
#     pd_means[count]=d_mean
    #Trends in dp and sp: fit each patient's data with a linear regression and record the slope.
    d_regr = linear_model.LinearRegression()
    d_regr.fit(p_df.egfr_date.reshape(-1, 1), p_df.egfr.reshape(-1, 1))
    b=d_regr.coef_.item(0)
    ps_trends[count]=b

x_egfr = pd.DataFrame({'patient_id':unique_ids, 
                     'egfr_mean': ps_means,'egfr_std': ps_std})

/Users/geenaildefonso/anaconda/envs/bios8366/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [85]:
x_egfr.head()

,egfr_mean,egfr_std,patient_id
0,87.336092,24.538705,50135262
1,51.882929,18.630883,50135361
2,84.715450,14.981761,50135369
3,35.632098,19.640734,50135375
4,35.230333,32.324784,50135425


### ICD9: 
Coding of diagnosed diseases and health problems. This is an international standard for classifying diseases, including nuanced classifications of a wide variety of signs, symptoms, abnormal findings, complaints, social circumstances, and external causes of injury or disease. 

1. "ICD9_Code" (code for each disease/health problem)
2. "Event_date"

This data is included as categorical data in our models.

In [9]:
###
#   ICD9
###
icd9 = pd.read_csv(data_path+"FONNESBECK_ICD9.csv", parse_dates=['Event_date'], infer_datetime_format=True)
icd9.head()

icd9_clean = (icd9
              .rename(columns={"RUID": "patient_id", 
                               "ICD9_Code": "icd9_code",
                               "Event_date": "icd9_event_date"}))
icd9_clean.head()

# no missing data
icd9_clean.isnull().sum()

patient_id         0
icd9_code          1
icd9_event_date    0
dtype: int64

### LAB: 
lab results, including the variables:
1. "Lab_name" (Abbreviated name of lab test)
2. "Lab_date" (date)
3. "Lab_value" (result of test; may be numerical or categorical, such as blood type)

This data was incorporated into our large dataset as categorical data. This information is hard (and somewhat unneccessary) to add to our models; for example, we do not expect blood type to have an effect on rate of readmission. For this reason, the data was not investigated as much as the other datasets.

In [10]:
###
#   LAB
###
lab = pd.read_csv(data_path+"FONNESBECK_LAB.csv", parse_dates=['Lab_date'], 
                  infer_datetime_format=True, quoting=csv.QUOTE_NONE, na_values=['>'])
lab.head()

lab_clean = (lab
             .rename(columns={"RUID": "patient_id", 
                              "Lab_name": "short_lab_name",
                              "Lab_date": "lab_date", 
                              "Lab_value": "lab_value"}))
lab_clean.head()

# decent number of missing lab values
# may be able to impute missing values if same patient 
lab_clean.isnull().sum()

patient_id            0
short_lab_name        4
lab_date              0
lab_value         18489
dtype: int64

### Phenotype: 
Patient attributes, including sex, race, and dates of birth and death.
Includes the variables: 
1. "Sex" (F or M) 
2. "DOB" (date of birth)
3. "DOD" (date of death)
4. "Race" (W = white, B = black, A = Asian, N = Native American, H = hispanic, U = unidentified). 

Cleaning of the Phenotype dataset:

In [42]:

###
#   PHENOTYPE
###
phenotype = pd.read_csv(data_path+"df_pheno_clean.csv", parse_dates=['DOB'], infer_datetime_format=True)
phenotype.head()

# phenotype_clean = (phenotype
#                    .rename(columns={"RUID": "patient_id", 
#                                     "Sex": "sex",
#                                     "DOB": "DOB", 
#                                     "DOD": "DOD",
#                                     "Race": "race"})
#                    .replace({'sex':  {'F': 0,'M': 1, 'U': 'NaN', '.': 'NaN', 'NA': 'NaN'}}))
# phenotype_clean.head()

# lots of missing DOBs and sex
phenotype.isnull().sum()
df_pheno = phenotype[['patient_id','sex', 'DOB', 'race']]

## Creating the X dataset for prediction: Variables of Interest

After all of the above is done, we have each cleaned and organized dataset, and the ADT data is arranged into a dataframe with patient ID, admission date, and discharge date. Thus, each patient ID may be repeated for multiple rows, depending on how often they were admitted, but each visit should only be recorded once in its own row. To make a large dataset with all of the information we want to include for modeling, we will add data from the other datasets using the dates in those data sets. We will parse the other data sets (for example, BMI), by date (Date_BMI). For each event in the BMI dataset, we will find the matching patient by ID. We will then find the correct column by comparing the date of the measurement (Date_BMI) to the Admission date and Discharge date in each row for that patient (i.e., adding the BMI measurement to the column where the patient ID matches and Admission_date $\leq$ Date_BMI $\leq$ Discharge_date.)

In [86]:
merge1 = pd.merge(x_bmi,x_bp, how='outer', on = 'patient_id')
merge2 = pd.merge(x_egfr, df_pheno,how='outer', on = 'patient_id')
merged = pd.merge(merge1, merge2, how = 'outer', on ='patient_id')
merged.head()
# merged.to_csv('merged.csv')
# df_merged = pd.concat([x_bmi,x_bp, x_egfr, df_pheno], join = 'outer')
# df_merged.head()

,bmi_mean,bmi_std,patient_id,pd_mean,pd_std,ps_mean,ps_std,egfr_mean,egfr_std,sex,DOB,race
0,43.443846,4.362163,50135262,64.086464,13.579633,141.160834,20.377205,87.336092,24.538705,0.0,1949-09-20,W
1,40.104062,56.286141,50135361,59.877885,12.678492,112.785374,18.694088,51.882929,18.630883,1.0,1932-02-15,W
2,29.091786,26.354793,50135369,70.011200,11.986641,115.893943,17.587347,84.715450,14.981761,1.0,1958-05-04,W
3,26.887687,2.139820,50135375,67.844574,12.547771,118.336822,21.254275,35.632098,19.640734,1.0,1943-05-01,B
4,33.648269,5.109400,50135425,56.765079,9.300600,128.038095,16.444496,35.230333,32.324784,0.0,1946-10-02,W


In [89]:
values = {'bmi_mean': merged.bmi_mean.mean(), 'bmi_std': merged.bmi_std.mean(), 'pd_mean': merged.pd_mean.mean(),
         'pd_std':merged.pd_std.mean(), 'ps_std': merged.ps_std.mean(), 'ps_mean': merged.ps_mean.mean(), 
         'egfr_mean': merged.egfr_mean.mean(), 'egfr_std': merged.egfr_std.mean()}
values = merged.fillna(value = values)
values.to_csv('merged_df.csv')

In [79]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
merged = imp.fit(merged.bmi_mean.reshape(-1, 1))

/Users/geenaildefonso/anaconda/envs/bios8366/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [80]:
merged.transform()

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

## Creating the Y variable dataset for prediction: Rate of Readmission

We used the same cleaned and organized ABT dataset to create a "Y" dataset that will be used to check our predictions of readmissions within 30 days of discharge. For each patient, we ran a for loop through the rows. Each row has a different discharge date, and this date was compared to all of the other admission dates to see if any admission dates were within 30 days of the discharge. If there is such a date, the loop is broken, the patient received a "1" value for a "readmitted" variable, and the next patient is investigated. This will give us a binary variable corresponding to the patients that were readmitted within 30 days at least once at any time. When building our models, we can use this information to check the accuracy of our models and for cross-validation.

In [13]:
x = pd.read_csv("df_adt_clean.csv", parse_dates=['admission_date', 'discharge_date'], index_col=0)
x.head()

,patient_id,admission_date,discharge_date
0,50135262,2007-02-08,2007-02-12
1,50135262,2007-08-03,2007-08-06
2,50135262,2014-11-15,2014-11-18
3,50135262,2007-08-28,2007-08-29
4,50135262,2012-09-15,2012-09-22


Using the above dataframe, I can compare discharge dates and admission dates for each patient. I will write a loop below that looks patient by patient. For each patient, I will loop through the discharge dates and compare them to the admission dates. If any are within 30 days of discharge, I will break from the inner loop, give the patient a "1", and move to the next patient.

In [14]:
unique_ids = x.patient_id.unique()
unique_ids

array([50135262, 50135361, 50135369, ..., 53736421, 53736422, 53736423])

In [15]:
patients_readmitted = np.zeros(len(unique_ids))
print(len(unique_ids))
for p, count in zip(unique_ids, range(len(unique_ids))):
#for p in [50135262]:
#make a subsetted dataframe of just that ID
    p_df = x[x.patient_id == p]
    p_df=p_df.reset_index(drop=True)
    #for each discharge_date in that list:
    for i in range(len(p_df)):
    #compare the discharge date to all of the admission dates in list
        discharge = p_df.loc[i]["discharge_date"]
        for j in range(len(p_df)):
            readmit = p_df.loc[j]["admission_date"]
            if (readmit - discharge > pd.to_timedelta('0 days')) and (readmit - discharge < pd.to_timedelta('30 days')):
                patients_readmitted[count]=patients_readmitted[count]+1

7914


In [16]:
y = pd.DataFrame({'patient_id':unique_ids, 'readmission':patients_readmitted})

In [19]:
y.head()

,patient_id,readmission
0,50135262,1.0
1,50135361,7.0
2,50135369,2.0
3,50135375,10.0
4,50135425,1.0


Y is now a dataframe that includes the number of times the patient has been readmitted within 30 days, and the patient ID. We plan to model whether or not a patient was readmitted within 30 days at all, but we also though we may be able to predict more information by modeling a discrete variable, rather than a binary (1/0) variable.